In [ ]:
import sys
sys.path.append('..')
import pickle
#harry's code is named "policy"
with open('../brute_force_VI/brute_force_solution_2.pkl', 'rb') as f:
    policy = pickle.load(f)

#marks code
with open('../layered_VI/layered_sol.pkl', 'rb') as g:
    policy_mark = pickle.load(g)

In [10]:
# policy[(i,j,j)] i = opponents score, j = player score, k = turn total 
# 1 = roll again 
# 0 = stick 
policy_mark[(5,5,4)]

np.int64(1)

In [4]:
import random

In [211]:
#trying to simulate a game - policy is going to just be harrys one as he got that working

#dice roll
def roll_die():
    return random.randint(1,6)

def game_pig(policy_player_1, policy_player_2):
    scores = [0,0]
    player = 0 

    while max(scores) < 100:
        turn_total = 0
        while True:
            if (player == 0 and policy_player_1.get((scores[0], scores[1], turn_total), 0)) or \
               (player == 1 and policy_player_2.get((scores[1], scores[0], turn_total), 0)):
                roll = roll_die()
                if roll == 1:
                    turn_total = 0
                    break
                else:
                    turn_total += roll
            else:
                scores[player] += turn_total
                break

        player = 1 - player
    return 0 if scores[0] >= 100 else 1 
    

In [212]:
game_pig(policy, policy)


policy_mark[4,5,25]

np.int64(0)

In [213]:
def simulate_games(n, policy1, policy2):
    results = [0,0]
    for _ in range(n):
        winner = game_pig(policy1, policy2)
        results[winner] +=1
    return results

results_harry = simulate_games(10000, policy, policy)
print(f"Player 0 wins: {results_harry[0]}, Player 1 wins: {results_harry[1]}")

Player 0 wins: 4989, Player 1 wins: 5011


In [215]:
results_mark = simulate_games(10000, policy_mark, policy_mark)
print(f"Player 0 wins: {results_mark[0]}, Player 1 wins: {results_mark[1]}")

Player 0 wins: 5275, Player 1 wins: 4725


In [ ]:
#used sophies policy from the pig competition

policy_sophie = {
    (my_score, op_score, round_score): (
        False if my_score + round_score > 100
        else round_score < 21 + round((op_score - my_score) / 8) + 1
             or my_score >= 71
             or op_score >= 71
    )
    for my_score in range(101)
    for op_score in range(101)
    for round_score in range(101)
}
policy_sophie[4,4,5]

game_pig(policy_sophie, policy_sophie)

1

In [218]:
results_sophie = simulate_games(1000, policy_sophie, policy_mark)
print(f"Player 0 wins: {results_sophie[0]}, Player 1 wins: {results_sophie[1]}")


Player 0 wins: 661, Player 1 wins: 339


In [207]:
policy_hold_at_20 = {
    (my_score, op_score, round_score): (
        False if round_score >= 20 or my_score + round_score > 100 else True
    )
    for my_score in range(101)
    for op_score in range(101)
    for round_score in range(101)
}

policy_hold_at_20[4, 4,21]

False

In [206]:
results_hold_20 = simulate_games(1000, policy_mark, policy_hold_at_20)
print(f"Player 0 wins: {results_hold_20[0]}, Player 1 wins: {results_hold_20[1]}")


Player 0 wins: 586, Player 1 wins: 414
